# Converts the JHU Cumulative Case Count Data to Daily Case Count Data

# TO DO:

- [X] Write the code that lets you convert the US data to long
- [X] Write the code that converts the column names in the Oxford data set to match the column names in the JHU dat
- [ ] Write the code that merges the countries to their offical alpha 3 code in the JHU: **[Link to Codes](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes)**
- [ ] **ALTERNATIVE TO ABOVE** use python-Levenshtein [Docs](https://rawgit.com/ztane/python-Levenshtein/master/docs/Levenshtein.html) distance to match similar country names 
> would still need to pair the high probability matches 
- [ ] Need to explore the missingness of the Oxford data. Sort the countries by GDP and examine what the missingness matrix looks like. **If you could run imputation on this data then you would have a major leg up on the other paper working on the similar topic. (on to of the other benefits to your paper)**
- [ ] Write the code that merges in the time series data for the diffent control measures
- [ ] Write the code transposes the combined data with the control measures included
- [ ] Write the code merges in the Country Safety Index data



## Importing the JHU COVID Case Data

In [1]:
import pandas as pd
import os
import sys
data_path = r'..\csse_covid_19_data\csse_covid_19_time_series'
out_data_path = r'..\Modified Data Sets'

case_pre = pd.read_csv(f'{data_path}/time_series_covid19_confirmed_global.csv')
death_pre = pd.read_csv(f'{data_path}/time_series_covid19_deaths_global.csv')


# case_pre_us = pd.read_csv(f'{data_path}/time_series_covid19_confirmed_US.csv')
# death_pre_us = pd.read_csv(f'{data_path}/time_series_covid19_deaths_US.csv')

import shutil
file_list = os.listdir(out_data_path)
for files in file_list:
    if files.find('.xlsx') >= 0:
        shutil.move(f'{out_data_path}/{files}',f'{out_data_path}/ARCHIVE/{files}')
def DF_Transform(df, outcome):
    global data_path
    global out_data_path
    
    # Data Cleaning
    df.drop(labels={'Lat','Long'},axis=1, inplace = True)
    df.loc[df['Country/Region'].str.contains('Congo'), 'Country/Region'] ='Congo'
    df.loc[df['Country/Region'].str.contains('Korea, South'), 'Country/Region']= 'South Korea'
    
    # Data Manipulation
    df = df.groupby(by='Country/Region').sum().T.apply(lambda x: x-x.shift(1),axis=0)
    df.rename(columns={'Country/Region':'Date'},inplace=True)
    df.columns = df.columns.str.replace(' ','_').str.lower()
    df = df.reset_index()
    df.rename(columns={'index':'date'},inplace= True)
    df.date = pd.to_datetime(df.date).dt.date
#     df['var'] = outcome[16:]
    
    filename = f'{out_data_path}/{outcome} (Through {df.date.max()}).xlsx'
    df.to_excel(filename, index=False)
    return df
case = DF_Transform(case_pre, 'Global COVID-19 Case Count')
death = DF_Transform(death_pre, 'Global COVID-19 Death Count')
# case = DF_Transform(case_pre_us, 'US COVID-19 Case Count')
# death = DF_Transform(death_pre_us, 'US COVID-19 Death Count')

In [7]:
def long_maker(dset, var):
    name_list = dset.columns.to_list()[1:]
    long_df = pd.DataFrame(columns = {'date', var, 'country'})
    for name in name_list:
        df = dset.filter(items={name, 'date'})
        df['country'] = name
        df.rename(columns={name:var},inplace = True)
        long_df = pd.concat([long_df, df],axis=0)
    long_df.date = pd.to_datetime(long_df.date)
    return long_df.reset_index(drop=True)

In [8]:
long_case = long_maker(case, 'case_count')
long_death = long_maker(death, 'death_count')

## Working on the Oxford Data Set

In [4]:
def Correcting_Col_Names(infile_path, dset):
    infile = pd.read_csv(infile_path)
    # og_col_list = infile.columns.to_list()
    df = pd.DataFrame(infile.columns.to_list())
    df['col'] = df[2:].apply(lambda x: pd.to_datetime(x).dt.strftime('X%m/X%d/%Y').str.replace('X0','').str.replace('X',''))
    df['col'][0] = 'country'
    df['col'][1] = 'country_code'
    cols = df.col.to_list()
    control = pd.read_csv(infile_path, names = cols, skiprows={0:1})
    indexNames = control[ control['country_code'].isna()].index
    control.drop(indexNames, inplace=True)
    df = controls_transpose(control, dset)
    return df

In [5]:
def controls_transpose(dset, var):
    df = dset.T
    df.columns = df.iloc[0]
    df.columns =df.columns.str.lower()
    df.drop(axis=0, index = {'country', 'country_code'},inplace = True)
    df = df.reset_index().rename(columns={'index':'date'})
    df = long_maker(df, var)
    return df

In [22]:
c_measures_path = r'../../../covid-policy-tracker/data'
school = Correcting_Col_Names(f'{c_measures_path}/timeseries/c1_schoolclosing.csv', 'school_close')
work = Correcting_Col_Names(f'{c_measures_path}/timeseries/c2_workplaceclosing.csv', 'work_close')
pub_events = Correcting_Col_Names(f'{c_measures_path}/timeseries/c3_cancelpublicevents.csv', 'public_events')
gatherings = Correcting_Col_Names(f'{c_measures_path}/timeseries/c4_restrictionsongatherings.csv', 'large_gather')
pub_transpo = Correcting_Col_Names(f'{c_measures_path}/timeseries/c5_closepublictransport.csv', 'public_transpo')
stay_home = Correcting_Col_Names(f'{c_measures_path}/timeseries/c6_stayathomerequirements.csv' ,'stay_home')
domestic_travel = Correcting_Col_Names(f'{c_measures_path}/timeseries/c7_domestictravel.csv' ,'domestic_travel')
int_travel = Correcting_Col_Names(f'{c_measures_path}/timeseries/c8_internationaltravel.csv' ,'internat_travel')

In [23]:
school.date.max()

Timestamp('2020-09-22 00:00:00')

In [24]:
long_df = long_case.merge(long_death, on=['date', 'country'])
long_df = long_df.merge(school, on=['date', 'country'],how='outer')
long_df = long_df.merge(work, on=['date', 'country'],how='outer')
long_df = long_df.merge(pub_events, on=['date', 'country'],how='outer')
long_df = long_df.merge(gatherings, on=['date', 'country'],how='outer')
long_df = long_df.merge(pub_transpo, on=['date', 'country'],how='outer')
long_df = long_df.merge(stay_home, on=['date', 'country'],how='outer')
long_df = long_df.merge(domestic_travel, on=['date', 'country'],how='outer')
long_df = long_df.merge(int_travel, on=['date', 'country'],how='outer')

In [25]:
long_df.sort_values(by=['country', 'date'])

,date,case_count,country,death_count,school_close,work_close,public_events,large_gather,public_transpo,stay_home,domestic_travel,internat_travel
45707,2020-01-01,NaN,afghanistan,NaN,0,0,0,0,0,0,0,0
45708,2020-01-02,NaN,afghanistan,NaN,0,0,0,0,0,0,0,0
45709,2020-01-03,NaN,afghanistan,NaN,0,0,0,0,0,0,0,0
45710,2020-01-04,NaN,afghanistan,NaN,0,0,0,0,0,0,0,0
45711,2020-01-05,NaN,afghanistan,NaN,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
45438,2020-09-18,14.0,zimbabwe,0.0,.,.,.,.,.,.,.,.
45439,2020-09-19,25.0,zimbabwe,1.0,.,.,.,.,.,.,.,.
45440,2020-09-20,11.0,zimbabwe,0.0,.,.,.,.,.,.,.,.
60872,2020-09-21,NaN,zimbabwe,NaN,.,.,.,.,.,.,.,.


In [ ]:
long_df2

In [ ]:
test

## trying to work out country linking stuff

In [ ]:
control_measures = pd.read_csv(f'{c_measures_path}/OxCGRT_latest_allchanges.csv')
control_measures.columns = control_measures.columns.str.lower()

In [ ]:
ox_country = control_measures.countryname.unique()

In [ ]:
jhu_country = case_pre['Country/Region']

In [ ]:
ox_country.shape

In [ ]:
jhu_country.shape

In [ ]:
from fuzzywuzzy import fuzz,process
from Levenshtein import distance,ratio
import pandas as pd
jhu = []
similar = []
ox = []
for countries in ox_country:
    results = process.extractOne(countries,list(jhu_country)) 
    if results[1] < 100:
        jhu.append(results[0])
        similar.append(results[1])
        ox.append(countries)
output = pd.DataFrame({'oxford_country':ox, 
                      'similarity':similar,
                       'jhu_country':jhu                       
                      })

In [ ]:
output.sort_values(by='similarity', ascending=False)

In [ ]:
df = pd.DataFrame(output)

In [ ]:
df

# Code Grave Yard

### Importing the Oxford Control Measures Data Set (Ordinal)